In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Project'

/content/drive/.shortcut-targets-by-id/11CaULD0eo2bzOUTXLY-_ejaxtRENy7I7/Project


In [3]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Successfully uninstalled keras-2.7.0
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0


In [4]:
!pip install tensorflow-gpu==1.13.1
!pip install keras==2.1.0
!pip install h5py==2.10.0

     |████████████████████████████████| 345.0 MB 4.0 kB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 367 kB 46.3 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 3.2 MB 15.7 MB/s 
     |████████████████████████████████| 4.1 MB 37.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
     |████████████████████████████████| 302 kB 4.3 MB/s 
     |████████████████████████████████| 2.9 MB 4.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.6.0
    Uninstalling h5py-3.6.0:
      Successfully uninstalled h5py-3.6.0


In [5]:
import torch
import os
import cv2
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import albumentations
import albumentations.pytorch
import json
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
import sys
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import skimage.draw
import numpy as np
from array import *

import albumentations
import albumentations.pytorch

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
# COCO_WEIGHTS_PATH = os.path.join("/content/drive/MyDrive/Project", "mask_rcnn_coco.h5")

COCO_WEIGHTS_PATH = os.path.join("/content/drive/MyDrive/Project", "logs/object20211128T0634/mask_rcnn_object_0593.h5")

DEFAULT_LOGS_DIR = os.path.join("/content/drive/MyDrive/Project", "logs")

In [8]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + phone,laptop and mobile

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640

    # VALIDATION_STEPS = 10

In [9]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "burr")
        self.add_class("object", 2, "groove")
        self.add_class("object", 3, "diagonal")
        # self.add_class("object", 2, "Man")

        self.image_path = []

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        self.annotations1 = []
        # path = os.path.join("/content/drive/MyDrive/Project/Data/train","Path.txt")
        path = os.path.join(dataset_dir,"Path2.txt")
        path = open(path)
        for x in path:
            x = x.replace('\n', '')
            if x.lower().endswith(('.jpg', '.png', '.jpeg')):
                self.image_path.append(x)
            elif x.lower().endswith('.json'):
                self.annotations1.append(x)

        print("images = ",len(self.image_path))
        print("Annot = ",len(self.annotations1))
        
        # Add images
        for i in range(len(self.image_path)-6):
            x = self.annotations1[i]
            with open(x) as f:
                a = json.load(f)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [(np.stack(r['points'], axis=1)).tolist() for r in a['shapes']] 
            objects = [s['label'] for s in a['shapes']]
            print("Index = ", i)
            name_dict = {"burr": 1,"groove": 2 ,"diagonal": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            # print("numids",num_ids)
            image_path = self.image_path[i]
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['imagePath'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	# rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            rr, cc = skimage.draw.polygon(p[1], p[0])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [10]:
def train(model):
# def train():
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/drive/MyDrive/Project/Data", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/drive/MyDrive/Project/Data", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=600,
                layers='heads')

In [11]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)
# train()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Re-starting from epoch 593
images =  700
Annot =  700
Index =  0
Index =  1
Index =  2
Index =  3
Index =  4
Index =  5
Index =  6
Index =  7
Index =  8
Index =  9
Index =  10
Index =  11
Index =  12
Index =  13
Index =  14
Index =  15
Index =  16
Index =  17
Index =  18
Index =  19
Index =  20
Index =  21
Index =  22
Index =  23
Index =  24
Index =  25
Index =  26
Index =  27
Index =  28
Index =  29
Index =  30
Index =  31
Index =  32
Index =  33
Index =  34
Index =  35
Index =  36
Index =  37
Index =  38
Index =  39
Index =  40
Index =  41
Index =  42
Index =  43
Index =  44
Index =  45
Index =  46
Index =  47
Index =  48
Index =  49
Index =  50
Index =  51
Index =  52
Index =  53
Index =  54
Index =  55
Index =  56
Index =  57
Index =  58
Index =  59
Index =  60
Index 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 594/600
100/100 [==============================] - 241s 2s/step - loss: 0.7128 - rpn_class_loss: 7.7912e-04 - rpn_bbox_loss: 0.0245 - mrcnn_class_loss: 0.1666 - mrcnn_bbox_loss: 0.1741 - mrcnn_mask_loss: 0.3468 - val_loss: 3.0578 - val_rpn_class_loss: 0.5207 - val_rpn_bbox_loss: 0.8155 - val_mrcnn_class_loss: 0.5194 - val_mrcnn_bbox_loss: 0.4130 - val_mrcnn_mask_loss: 0.7892
Epoch 595/600
 49/100 [=============>................] - ETA: 1:06 - loss: 0.2034 - rpn_class_loss: 0.0019 - rpn_bbox_loss: 0.0234 - mrcnn_class_loss: 0.0294 - mrcnn_bbox_loss: 0.0515 - mrcnn_mask_loss: 0.0971

ERROR:root:Error processing image {'id': 'frame_64.jpg', 'source': 'object', 'path': '/content/drive/MyDrive/Project/Semantic segmentation/Good/Diagonal/frames_compress21_D/frame_64.jpg', 'width': 1000, 'height': 1000, 'polygons': [], 'num_ids': []}
Traceback (most recent call last):
  File "/content/drive/.shortcut-targets-by-id/11CaULD0eo2bzOUTXLY-_ejaxtRENy7I7/Project/mrcnn/model.py", line 1737, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/drive/.shortcut-targets-by-id/11CaULD0eo2bzOUTXLY-_ejaxtRENy7I7/Project/mrcnn/model.py", line 1293, in load_image_gt
    class_ids = class_ids[_idx]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 512


100/100 [==============================] - 168s 2s/step - loss: 0.1759 - rpn_class_loss: 0.0014 - rpn_bbox_loss: 0.0202 - mrcnn_class_loss: 0.0240 - mrcnn_bbox_loss: 0.0410 - mrcnn_mask_loss: 0.0894 - val_loss: 3.4372 - val_rpn_class_loss: 0.5862 - val_rpn_bbox_loss: 0.9163 - val_mrcnn_class_loss: 0.6257 - val_mrcnn_bbox_loss: 0.4121 - val_mrcnn_mask_loss: 0.8969
Epoch 596/600
100/100 [==============================] - 167s 2s/step - loss: 0.1496 - rpn_class_loss: 7.0849e-04 - rpn_bbox_loss: 0.0167 - mrcnn_class_loss: 0.0197 - mrcnn_bbox_loss: 0.0292 - mrcnn_mask_loss: 0.0832 - val_loss: 3.2747 - val_rpn_class_loss: 0.5203 - val_rpn_bbox_loss: 0.7857 - val_mrcnn_class_loss: 0.6475 - val_mrcnn_bbox_loss: 0.3998 - val_mrcnn_mask_loss: 0.9214
Epoch 597/600
 39/100 [==========>...................] - ETA: 1:19 - loss: 0.1593 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0184 - mrcnn_class_loss: 0.0252 - mrcnn_bbox_loss: 0.0291 - mrcnn_mask_loss: 0.0855

ERROR:root:Error processing image {'id': 'frame_64.jpg', 'source': 'object', 'path': '/content/drive/MyDrive/Project/Semantic segmentation/Good/Diagonal/frames_compress21_D/frame_64.jpg', 'width': 1000, 'height': 1000, 'polygons': [], 'num_ids': []}
Traceback (most recent call last):
  File "/content/drive/.shortcut-targets-by-id/11CaULD0eo2bzOUTXLY-_ejaxtRENy7I7/Project/mrcnn/model.py", line 1737, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/drive/.shortcut-targets-by-id/11CaULD0eo2bzOUTXLY-_ejaxtRENy7I7/Project/mrcnn/model.py", line 1293, in load_image_gt
    class_ids = class_ids[_idx]
IndexError: boolean index did not match indexed array along dimension 0; dimension is 0 but corresponding boolean dimension is 512


100/100 [==============================] - 168s 2s/step - loss: 0.1505 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0181 - mrcnn_class_loss: 0.0218 - mrcnn_bbox_loss: 0.0268 - mrcnn_mask_loss: 0.0827 - val_loss: 3.3284 - val_rpn_class_loss: 0.5039 - val_rpn_bbox_loss: 0.8077 - val_mrcnn_class_loss: 0.6342 - val_mrcnn_bbox_loss: 0.4193 - val_mrcnn_mask_loss: 0.9633
Epoch 598/600
100/100 [==============================] - 167s 2s/step - loss: 0.1475 - rpn_class_loss: 9.4824e-04 - rpn_bbox_loss: 0.0176 - mrcnn_class_loss: 0.0202 - mrcnn_bbox_loss: 0.0258 - mrcnn_mask_loss: 0.0830 - val_loss: 3.5091 - val_rpn_class_loss: 0.5805 - val_rpn_bbox_loss: 0.8968 - val_mrcnn_class_loss: 0.6738 - val_mrcnn_bbox_loss: 0.4165 - val_mrcnn_mask_loss: 0.9414
Epoch 599/600
100/100 [==============================] - 169s 2s/step - loss: 0.1432 - rpn_class_loss: 9.1562e-04 - rpn_bbox_loss: 0.0166 - mrcnn_class_loss: 0.0209 - mrcnn_bbox_loss: 0.0244 - mrcnn_mask_loss: 0.0804 - val_loss: 3.4233 - val_rpn_clas